In [1]:
import pandas as pd
from environments.forex_tradning import tgym

C:\github\TradingGym\tvenv\lib\site-packages\setuptools\distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(


In [2]:
parameters = {
"observation_list":["_dt","open","high","low","close","minute","hour","weekday","week","month","year","RSI","SMA"],
}
parameters = {
"observation_list":["open","high","low","close","minute","hour","weekday","RSI","SMA"],
"stop_loss_max":1000
}
file = "./data/split/GBPUSD/weekly/GBPUSD_20191110_W_148.csv"
file = "./data/split/GBPUSD/weekly/GBPUSD_20191208_W_1523.csvx"
df = pd.read_csv(file)
t = tgym(df, **parameters)

initial done:
             observation_list:['open', 'high', 'low', 'close', 'minute', 'hour', 'weekday', 'RSI', 'SMA']
             assets:['GBPUSD' 'ABCD']
             time serial: 2/12/2019 0:00 -> 6/12/2019 9:55 length: 1440


In [3]:
import time
import datetime
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env

# if model: del model # remove to demonstrate saving and loading
model_name='./data/models/GBPUSD-week-20211019133954'
model = PPO.load(model_name)

start_time = time.time()
obs = t.reset()
t.current_step=0
done = False
while not done:
    action, _states = model.predict(obs)
    obs, rewards, done, info = t.step(action)
    t.render(mode='human')
print(f"--- running time: {(time.time() - start_time)}---")

  TicketSymbol  Type              ActionTime                                 ActionPrice   MaxDD              CloseTimeClosePrice                                     Reward  DateDuration        Status  

       1 GBPUSD      0 2/12/2019 0:05                       1.10214        0 2/12/2019 0:10   1.10214                      -10                    2        1
       2 ABCD        0 2/12/2019 0:05                       1.10572        0 2/12/2019 0:10   1.10572                      -10                    2        1

       3 GBPUSD      0 2/12/2019 0:10                       1.10206        0 2/12/2019 0:15   1.10206                      -10                    2        1
       4 ABCD        0 2/12/2019 0:10                       1.10564        0 2/12/2019 0:15   1.10564                      -10                    2        1

       5 GBPUSD      0 2/12/2019 0:15                       1.10207        0 2/12/2019 0:20   1.10207                      -10                    2        1
       6 

In [9]:
import time
import datetime
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env

# Parallel environments

start_time = time.time()
model_name = "./data/models/GBPUSD-week-" + datetime.datetime.now().strftime('%Y%m%d%H%M%S')
print(f'save model as {model_name}')
model = PPO("MlpPolicy", t, verbose=1)
model.learn(total_timesteps=100000)
model.save(model_name)
print(f"--- trainning time: {(time.time() - start_time)}---")


save model as ./data/models/GBPUSD-week-20211019133954
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 130       |
|    ep_rew_mean     | -4.13e+03 |
| time/              |           |
|    fps             | 1166      |
|    iterations      | 1         |
|    time_elapsed    | 1         |
|    total_timesteps | 2048      |
----------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 129          |
|    ep_rew_mean          | -4.27e+03    |
| time/                   |              |
|    fps                  | 723          |
|    iterations           | 2            |
|    time_elapsed         | 5            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0014926351 |
|    clip_fraction        | 

Ticket  Symbol  Type    ActionTime                                           ActionPrice   MaxDD   CloseTime           ClosePrice                                     Reward  DateDuration        Status  

       1GBPUSD         0      2/12/2019 0:05                     1.10214       02/12/2019 0:10      1.10214                      -10                   2       1

       3GBPUSD         0      2/12/2019 0:10                     1.10206       02/12/2019 0:15      1.10206                      -10                   2       1
       4ABCD           0      2/12/2019 0:10                     1.10564       02/12/2019 0:15      1.10564                      -10                   2       1

       5GBPUSD         0      2/12/2019 0:15                     1.10207       02/12/2019 0:20      1.10207                      -10                   2       1
       6ABCD           0      2/12/2019 0:15                     1.10566       02/12/2019 0:20      1.10566                      -10                  

In [12]:
_header =f'{"Ticket":8}{"Symbol":8}{"Type":8}{"ActionTime":20} \
                    {"ActionPrice":14}{"MaxDD":8}{"CloseTime":20}{"ClosePrice":14} \
                    {"Reward":8}{"DateDuration":20}{"Status":8}\n'
print(_header)

Ticket  Symbol  Type    ActionTime                               ActionPrice   MaxDD   CloseTime           ClosePrice                         Reward  DateDuration        Status  



In [15]:
!python test.py >

In [16]:
!script_output=$(python test.py 2>&1)
!echo $script_output

'script_output' is not recognized as an internal or external command,
operable program or batch file.
$script_output
